In [1]:
import os
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import TextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate


os.chdir('/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec')
# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [2]:
import os
print(os.getcwd())

/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec


In [3]:
import pandas as pd

# file_path = ('data/movie_record_per_user.csv')
file_path='data/train_movie.csv'
file_path_test='data/test_movie.csv'

data = pd.read_csv(file_path)
data['movie_explain'][0]

"Girl, Interrupted (1999) (Drama) ratings: 4 // Back to the Future (1985) (Comedy|Sci-Fi) ratings: 5 // Titanic (1997) (Drama|Romance) ratings: 4 // Cinderella (1950) (Animation|Children's|Musical) ratings: 5 // Meet Joe Black (1998) (Romance) ratings: 3 // Last Days of Disco, The (1998) (Drama) ratings: 5 // Erin Brockovich (2000) (Drama) ratings: 4 // Christmas Story, A (1983) (Comedy|Drama) ratings: 5 // To Kill a Mockingbird (1962) (Drama) ratings: 4 // One Flew Over the Cuckoo's Nest (1975) (Drama) ratings: 5 // Wallace & Gromit: The Best of Aardman Animation (1996) (Animation) ratings: 3 // Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi) ratings: 4 // Wizard of Oz, The (1939) (Adventure|Children's|Drama|Musical) ratings: 4 // Fargo (1996) (Crime|Drama|Thriller) ratings: 4 // Run Lola Run (Lola rennt) (1998) (Action|Crime|Romance) ratings: 4 // Rain Man (1988) (Drama) ratings: 5 // Saving Private Ryan (1998) (Action|Drama|War) ratings: 5 // Awakenings (

In [4]:
# 각 텍스트 데이터의 길이 계산
data['text_length'] = data['movie_explain'].apply(len)

# 평균 글자 수 계산
average_length = data['text_length'].mean()

# 결과 출력
print(f"평균 글자 수: {average_length}")


평균 글자 수: 9210.526158940398


In [ ]:
data['movie_explain'].mean()

In [30]:

# // 를 기준으로 자르기 
class CustomDelimiterTextSplitter(TextSplitter):
    def __init__(self, delimiter="//", chunk_size=500, chunk_overlap=200):
        self.delimiter = delimiter
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_text(self, text):
        sections = text.split(self.delimiter)
        chunks = []
        current_chunk = ""

        for section in sections:
            # 현재 청크에 텍스트를 추가할 수 있으면 추가
            if len(current_chunk) + len(section) + len(self.delimiter) <= self.chunk_size:
                if current_chunk:
                    current_chunk += self.delimiter + section
                else:
                    current_chunk = section
            else:
                # 청크가 가득 찬 경우 저장하고 초기화
                chunks.append(current_chunk.strip())
                current_chunk = section

        # 마지막 청크 추가
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

# CSV 파일 로드
loader = CSVLoader(file_path=file_path)
documents = loader.load()

# 각 행에 대해 청킹 수행 및 임베딩
text_splitter = CustomDelimiterTextSplitter(delimiter="//", chunk_size=500, chunk_overlap=200)
all_chunks = []  # 모든 유저의 청킹 결과를 저장할 리스트

for document in documents:
    page_content = document.page_content  # 현재 유저의 텍스트
    chunks = text_splitter.split_text(page_content)  # 유저별 청킹
    all_chunks.extend(chunks)  # 청킹된 모든 텍스트를 리스트에 추가

# 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 벡터 스토어 생성
vectorstore = FAISS.from_texts(all_chunks, embeddings)

# 벡터 스토어 저장 (옵션)
vectorstore.save_local("vectorstore_index_ratings_500")


- 벡터스토어 불러오기

In [23]:
# 동일한 임베딩 모델 설정 (저장 시 사용한 것과 동일해야 함)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 저장된 벡터스토어 불러오기
vectorstore = FAISS.load_local("vectorstore_index_ratings_500", embeddings, allow_dangerous_deserialization=True)


/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### test
 Transformers: The Movie, The (1986)Action|Animation|Children's|Sci-Fi|Thriller|War // How I Won the War (1967)Comedy|War // Peacemaker, The (1997)Action|Thriller|War // Devil's Own, The (1997)Action|Drama|Thriller|War // Heartbreak Ridge (1986)Action|War // Hot Shots! Part Deux (1993)Action|Comedy|War // Devil's Brigade, The (1968)War // Canadian Bacon (1994)Comedy|War // Renaissance Man (1994)Comedy|Drama|War // Heaven & Earth (1993)Action|Drama|War

In [5]:

len(data.iloc[105]['movie_explain'])
purchase_history=data.iloc[105]['movie_explain']
purchase_history

"Breakfast Club, The (1985) (Comedy|Drama) ratings: 4 // North by Northwest (1959) (Drama|Thriller) ratings: 4 // Home Alone (1990) (Children's|Comedy) ratings: 3 // Saving Private Ryan (1998) (Action|Drama|War) ratings: 5 // Little Shop of Horrors (1986) (Comedy|Horror|Musical) ratings: 4 // Cyrano de Bergerac (1990) (Action|Drama|Romance) ratings: 5 // Room with a View, A (1986) (Drama|Romance) ratings: 4 // Shop Around the Corner, The (1940) (Comedy|Romance) ratings: 4 // Singin' in the Rain (1952) (Musical|Romance) ratings: 5 // Shakespeare in Love (1998) (Comedy|Romance) ratings: 5 // Strictly Ballroom (1992) (Comedy|Romance) ratings: 5 // Roman Holiday (1953) (Comedy|Romance) ratings: 4 // Rebecca (1940) (Romance|Thriller) ratings: 4 // Like Water for Chocolate (Como agua para chocolate) (1992) (Drama|Romance) ratings: 4 // Persuasion (1995) (Romance) ratings: 5 // Broadcast News (1987) (Comedy|Drama|Romance) ratings: 4 // Gone with the Wind (1939) (Drama|Romance|War) ratings: 5 

In [36]:
df_test=pd.read_csv(file_path_test)
pd.set_option('display.max_colwidth', None)
df_test.iloc[105]

movie_explain    Starman (1984) (Adventure|Drama|Romance|Sci-Fi) ratings: 3
Name: 105, dtype: object

In [16]:
# query = f"Movies similar to those in this list: {purchase_history}"
# purchase_history=data.iloc[105]['movie_explain']
# # query = f"What movies do users watch after these: {purchase_history}?"

In [53]:
# query = f"Movies similar to those in this list: {purchase_history}"
purchase_history=data.iloc[105]['movie_explain']
query = " // ".join(purchase_history.split(" // ")[-10:])

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
record = retriever.get_relevant_documents(query)
# `record`에서 내용을 추출하여 요약
record_summary = "\n".join([doc.page_content for doc in record])
record_summary

"Sabrina (1954) (Comedy|Romance) ratings: 4 // Star Wars: Episode VI - Return of the Jedi (1983) (Action|Adventure|Romance|Sci-Fi|War) ratings: 4 // Last of the Mohicans, The (1992) (Action|Romance|War) ratings: 1 // Walking and Talking (1996) (Romance) ratings: 3 // Emma (1996) (Comedy|Drama|Romance) ratings: 4 // Living Out Loud (1998) (Comedy|Romance) ratings: 3 // Mrs. Brown (Her Majesty, Mrs. Brown) (1997) (Drama|Romance) ratings: 3 // Splash (1984) (Comedy|Fantasy|Romance) ratings: 4\nSabrina (1954) (Comedy|Romance) ratings: 5 // Cinema Paradiso (1988) (Comedy|Drama|Romance) ratings: 5 // Bell, Book and Candle (1958) (Comedy|Romance) ratings: 4 // Gay Divorcee, The (1934) (Comedy|Musical|Romance) ratings: 4 // American President, The (1995) (Comedy|Drama|Romance) ratings: 4 // Clueless (1995) (Comedy|Romance) ratings: 4 // English Patient, The (1996) (Drama|Romance|War) ratings: 5 // Chasing Amy (1997) (Drama|Romance) ratings: 4\nSabrina (1954) (Comedy|Romance) ratings: 5 // Rome

In [54]:
record

[Document(metadata={}, page_content='Sabrina (1954) (Comedy|Romance) ratings: 4 // Star Wars: Episode VI - Return of the Jedi (1983) (Action|Adventure|Romance|Sci-Fi|War) ratings: 4 // Last of the Mohicans, The (1992) (Action|Romance|War) ratings: 1 // Walking and Talking (1996) (Romance) ratings: 3 // Emma (1996) (Comedy|Drama|Romance) ratings: 4 // Living Out Loud (1998) (Comedy|Romance) ratings: 3 // Mrs. Brown (Her Majesty, Mrs. Brown) (1997) (Drama|Romance) ratings: 3 // Splash (1984) (Comedy|Fantasy|Romance) ratings: 4'),
 Document(metadata={}, page_content='Sabrina (1954) (Comedy|Romance) ratings: 5 // Cinema Paradiso (1988) (Comedy|Drama|Romance) ratings: 5 // Bell, Book and Candle (1958) (Comedy|Romance) ratings: 4 // Gay Divorcee, The (1934) (Comedy|Musical|Romance) ratings: 4 // American President, The (1995) (Comedy|Drama|Romance) ratings: 4 // Clueless (1995) (Comedy|Romance) ratings: 4 // English Patient, The (1996) (Drama|Romance|War) ratings: 5 // Chasing Amy (1997) (Dr

In [59]:
from langchain.prompts import ChatPromptTemplate

template = """
## item ## 
Here is the movie history list of user1: {query} 
## role ## 
Based on the other people's watching record: {record_summary}, recommend exactly 10 movies that user1 is likely to watch next. 
You must prioritize movies that appear directly in the given watching record summary ({record_summary}) and are not already in user1's history.
Do not recommend any movies that are already in user1's history {purchase_history}.
Ensure that the recommendations take into account movies from all parts of the {record_summary} (start, middle, and end), not just the initial sections.
Provide the recommendations as a numbered list.
"""

# 템플릿에서 ChatPromptTemplate 생성
prompt = ChatPromptTemplate.from_template(template)

# 템플릿 내용 확인
print(prompt)


input_variables=['purchase_history', 'query', 'record_summary'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['purchase_history', 'query', 'record_summary'], input_types={}, partial_variables={}, template="\n## item ## \nHere is the movie history list of user1: {query} \n## role ## \nBased on the other people's watching record: {record_summary}, recommend exactly 10 movies that user1 is likely to watch next. \nYou must prioritize movies that appear directly in the given watching record summary ({record_summary}) and are not already in user1's history.\nDo not recommend any movies that are already in user1's history {purchase_history}.\nEnsure that the recommendations take into account movies from all parts of the {record_summary} (start, middle, and end), not just the initial sections.\nProvide the recommendations as a numbered list.\n"), additional_kwargs={})]


In [60]:
record_summary

"Sabrina (1954) (Comedy|Romance) ratings: 4 // Star Wars: Episode VI - Return of the Jedi (1983) (Action|Adventure|Romance|Sci-Fi|War) ratings: 4 // Last of the Mohicans, The (1992) (Action|Romance|War) ratings: 1 // Walking and Talking (1996) (Romance) ratings: 3 // Emma (1996) (Comedy|Drama|Romance) ratings: 4 // Living Out Loud (1998) (Comedy|Romance) ratings: 3 // Mrs. Brown (Her Majesty, Mrs. Brown) (1997) (Drama|Romance) ratings: 3 // Splash (1984) (Comedy|Fantasy|Romance) ratings: 4\nSabrina (1954) (Comedy|Romance) ratings: 5 // Cinema Paradiso (1988) (Comedy|Drama|Romance) ratings: 5 // Bell, Book and Candle (1958) (Comedy|Romance) ratings: 4 // Gay Divorcee, The (1934) (Comedy|Musical|Romance) ratings: 4 // American President, The (1995) (Comedy|Drama|Romance) ratings: 4 // Clueless (1995) (Comedy|Romance) ratings: 4 // English Patient, The (1996) (Drama|Romance|War) ratings: 5 // Chasing Amy (1997) (Drama|Romance) ratings: 4\nSabrina (1954) (Comedy|Romance) ratings: 5 // Rome

In [61]:
from langchain_core.output_parsers import StrOutputParser


llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
chain = prompt | llm | StrOutputParser()
reesponse= chain.invoke({"purchase_history":purchase_history, "record_summary": record_summary, "query":query})

In [62]:
reesponse

"Based on the given watching record summary and user1's movie history, here are 10 movie recommendations for user1:\n\n1. **Cinema Paradiso (1988)** - Comedy|Drama|Romance\n2. **Bell, Book and Candle (1958)** - Comedy|Romance\n3. **Gay Divorcee, The (1934)** - Comedy|Musical|Romance\n4. **American President, The (1995)** - Comedy|Drama|Romance\n5. **Clueless (1995)** - Comedy|Romance\n6. **Romeo and Juliet (1968)** - Drama|Romance\n7. **Wings of the Dove, The (1997)** - Drama|Romance|Thriller\n8. **Next Stop, Wonderland (1998)** - Comedy|Drama|Romance\n9. **Titanic (1997)** - Drama|Romance\n10. **Shampoo (1975)** - Comedy|Romance\n\nThese recommendations are based on the preferences and ratings patterns observed in user1's history and the watching record summary provided."

In [47]:
import re
movie_titles = re.findall(r"\d+\.\s\*\*(.*?)\s\(\d{4}\)", reesponse)
print(movie_titles)

['Cinema Paradiso', 'Bell, Book and Candle', 'Gay Divorcee, The', 'American President, The', 'Clueless']


In [48]:
tset_answer=df_test.iloc[105]['movie_explain']
movie_name = re.match(r"^(.*?)\s\(\d{4}\)", tset_answer).group(1)
print(movie_name)

Starman


## 수정해야 할 부분
1. 청킹이 잘 되가고 있는지
2. retrieve가 잘 되고 있는지
3. generate가 잘 되고 있는지

In [ ]:
# query = f"Movies similar to those in this list: {purchase_history}"
purchase_history=data.iloc[105]['movie_explain']
query = " // ".join(purchase_history.split(" // ")[-10:])

# query = f"What movies do users watch after these: {purchase_history}?"
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
record = retriever.get_relevant_documents(query)
# `record`에서 내용을 추출하여 요약
record_summary = "\n".join([doc.page_content for doc in record])
record_summary

In [48]:
import re
import pandas as pd

# 결과를 저장할 리스트
results = []
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 1부터 100까지 반복
for idx in range(1, 30):
    # movie_explain 추출
    tset_answer = df_test.iloc[idx]['movie_explain']
    
    # movie_name 추출
    try:
        movie_name = re.match(r"^(.*?)\s\(\d{4}\)", tset_answer).group(1)
    except AttributeError:
        movie_name = None  # 매칭 실패 시 None으로 처리

    # 쿼리 생성
    purchase_history=data.iloc[idx]['movie_explain']
    query = " // ".join(purchase_history.split(" // ")[-10:])
    # query = f"Movies similar to those in this list: {purchase_history}"
    # query
    # 벡터스토어에서 관련 문서 검색
    record = retriever.get_relevant_documents(query)
    
    # 검색 결과 요약
    record_summary = "\n".join([doc.page_content for doc in record])

    # LLM에 입력
    reesponse = chain.invoke({"purchase_history": purchase_history, "record_summary": record_summary})
    
    # movie_title 추출
    movie_titles = re.findall(r"\d+\.\s\*\*(.*?)\s\(\d{4}\)", reesponse)
    
    # movie_name이 movie_titles에 포함되는지 여부
    is_included = movie_name in movie_titles if movie_name else False

    # 결과 저장
    results.append({
        "movie_title": movie_titles,
        "movie_name": movie_name,
        "is_included": is_included
    })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

In [42]:
purchase_history

"Breakfast Club, The (1985) (Comedy|Drama) ratings: 4 // North by Northwest (1959) (Drama|Thriller) ratings: 4 // Home Alone (1990) (Children's|Comedy) ratings: 3 // Saving Private Ryan (1998) (Action|Drama|War) ratings: 5 // Little Shop of Horrors (1986) (Comedy|Horror|Musical) ratings: 4 // Cyrano de Bergerac (1990) (Action|Drama|Romance) ratings: 5 // Room with a View, A (1986) (Drama|Romance) ratings: 4 // Shop Around the Corner, The (1940) (Comedy|Romance) ratings: 4 // Singin' in the Rain (1952) (Musical|Romance) ratings: 5 // Shakespeare in Love (1998) (Comedy|Romance) ratings: 5 // Strictly Ballroom (1992) (Comedy|Romance) ratings: 5 // Roman Holiday (1953) (Comedy|Romance) ratings: 4 // Rebecca (1940) (Romance|Thriller) ratings: 4 // Like Water for Chocolate (Como agua para chocolate) (1992) (Drama|Romance) ratings: 4 // Persuasion (1995) (Romance) ratings: 5 // Broadcast News (1987) (Comedy|Drama|Romance) ratings: 4 // Gone with the Wind (1939) (Drama|Romance|War) ratings: 5 

## Evaluation

In [49]:
results_df

,movie_title,movie_name,is_included
0,"[Arrival, The, Mimic, Cliffhanger, Young Guns,...",Armageddon,False
1,"[Murphy's Romance, Cocoon, Real Genius, Weird ...","Little Mermaid, The",False
2,"[Rocky IV, Fugitive, The, Untouchables, The, R...",Rocky,False
3,"[Mission: Impossible, Escape from New York, Di...",Natural Born Killers,False
4,"[Sleepless in Seattle, You've Got Mail, Nottin...",Pretty Woman,False
5,"[Backdraft, Client, The, GoldenEye, True Lies,...",Backdraft,True
6,"[Bird on a Wire, Fly Away Home, Walk in the Cl...","Bodyguard, The",False
7,"[Willy Wonka and the Chocolate Factory, Beauty...",Antz,False
8,"[Up Close and Personal, Hot Shots! Part Deux, ...",Hero,False
9,"[Buena Vista Social Club, Trekkies, Stop Makin...",Mr. Death: The Rise and Fall of Fred A. Leucht...,False


### HIT@5

In [51]:
import pandas as pd

# CSV 파일 읽기
df =results_df

# Hit@k 계산 (k=5)
k = 5
df['hit'] = df.apply(lambda row: row['movie_name'] in row['movie_title'][:k], axis=1)

# 전체 Hit@k 계산
hit_at_k = df['hit'].mean()  # Hit@k 비율
print(f"Hit@{k}: {hit_at_k}")


Hit@5: 0.4827586206896552
